In [ ]:
#@title Cuadro resumen, donde se muestran las tablas e histogramas estadisticos de los angulos, ademas de los giros de las fotos.
from google.colab import drive
drive.mount('/content/drive/')

import os
import json
import pandas as pd
import numpy as np


path="/content/drive/My Drive/HAINTECH.CL/ejemplos/"
lista=pd.Series(os.listdir(path))
lista=lista[lista.str.endswith("json")].tolist()
for file in lista:
    A=pd.read_fwf(path+file,header=None,infer_nrows=10**9).fillna("")
    B=json.loads(A.sum(1).sum())
    C=pd.DataFrame.from_dict(B['textAnnotations'])
    F=C["description"]
    D=pd.DataFrame.from_dict(C.boundingPoly.tolist())
    D=pd.DataFrame.from_dict(D.vertices.tolist())
    x,y=D.iloc[0,0].values()
    for col in D.columns:
        E=pd.DataFrame.from_dict(D[col].tolist())-np.array([x,y])
        E.columns+=str(col)
        F=pd.concat([F,E],axis=1)
    angulo=np.median(np.arctan2(F["y1"]-F["y0"],F["x1"]-F["x0"]))
    X=dict()
    Y=dict()
    for col in D.columns:
      X[col]=np.cos(angulo)*F["x"+str(col)]-np.sin(angulo)*F["y"+str(col)]
      Y[col]=np.sin(angulo)*F["x"+str(col)]+np.cos(angulo)*F["y"+str(col)]
    X=pd.DataFrame(X)
    Y=pd.DataFrame(Y)
    X+=x-X.min(None).min(None)
    Y+=y-Y.min(None).min(None)
    Z=np.hypot(X,Y).idxmin(1)
    X=X.astype(int)
    Y=Y.astype(int)
    r=np.arange(4)
    if any(Z>0):
      for j in Z[Z>0].index:
        X.iloc[j]=X.iloc[j,(r-Z.iloc[j])%4]
        Y.iloc[j]=Y.iloc[j,(r-Z.iloc[j])%4]
    G=dict()
    for col in D.columns:
      G[col]=pd.DataFrame({"x":X[col],"y":Y[col]}).to_dict('records')
    G=pd.DataFrame(G).apply(lambda x: {'vertices':list(x)},axis=1).to_frame().rename(columns={0:'boundingTurnPoly'})
    G=pd.concat([C[['boundingPoly','description']],G],axis=1).to_dict("result")
    for k in C.index[C['locale'].notna()]:
      G[k].update({"locale":C['locale'][k]})
    B['textAnnotations']=G
    B.update({"angle":angulo*180/np.pi})
    #with open(path+"rotated/"+file.replace(".json","_rotated.json"), 'w') as outfile:
    #  outfile.write(json.dumps(B)) 

In [ ]:
Z.values